In [ ]:
# 🛠️ SETUP
!pip install -q ultralytics opencv-python tqdm

In [ ]:

import cv2
import os
import json
from ultralytics import YOLO
from tqdm import tqdm
from google.colab import files
from IPython.display import display, Image
from google.colab import drive


In [ ]:
# 📁 CONFIG
FRAME_OUTPUT_DIR = 'frames'
COCO_OUTPUT_PATH = 'detections.json'
FRAME_STEP = 30
MODEL_NAME = 'yolov8n.pt'


In [ ]:
# 📤 Upload your own video
uploaded = files.upload()
for fname in uploaded:
    VIDEO_PATH = fname

In [ ]:
# 🎞️ EXTRACT FRAMES
os.makedirs(FRAME_OUTPUT_DIR, exist_ok=True)
cap = cv2.VideoCapture(VIDEO_PATH)
frame_idx, saved_idx = 0, 0

print("Extracting frames...")
while True:
    success, frame = cap.read()
    if not success:
        break
    if frame_idx % FRAME_STEP == 0:
        cv2.imwrite(f"{FRAME_OUTPUT_DIR}/frame_{saved_idx:05d}.jpg", frame)
        saved_idx += 1
    frame_idx += 1

cap.release()
print(f"Saved {saved_idx} frames to {FRAME_OUTPUT_DIR}/")

In [ ]:
# 🤖 PRE-TAG WITH YOLO
model = YOLO(MODEL_NAME)
image_files = sorted([f for f in os.listdir(FRAME_OUTPUT_DIR) if f.endswith('.jpg')])

coco_output = {
    "images": [],
    "annotations": [],
    "categories": []
}
category_map = {}
next_image_id = 1
next_ann_id = 1
next_category_id = 1

for image_file in tqdm(image_files, desc="Pretagging"):
    img_path = os.path.join(FRAME_OUTPUT_DIR, image_file)
    results = model(img_path)[0]
    height, width = results.orig_shape

    coco_output["images"].append({
        "id": next_image_id,
        "file_name": image_file,
        "height": height,
        "width": width
    })

    for det in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls_id = det
        cls_id = int(cls_id)
        label = model.names[cls_id]

        if label not in category_map:
            category_map[label] = next_category_id
            coco_output["categories"].append({
                "id": next_category_id,
                "name": label
            })
            next_category_id += 1

        coco_output["annotations"].append({
            "id": next_ann_id,
            "image_id": next_image_id,
            "category_id": category_map[label],
            "bbox": [x1, y1, x2 - x1, y2 - y1],
            "area": (x2 - x1) * (y2 - y1),
            "iscrowd": 0
        })
        next_ann_id += 1

    next_image_id += 1

In [ ]:
# 💾 SAVE ANNOTATIONS
with open(COCO_OUTPUT_PATH, 'w') as f:
    json.dump(coco_output, f, indent=2)

print(f"COCO-format annotations saved to {COCO_OUTPUT_PATH}")

In [ ]:

# 📥 OPTIONAL: Download output
files.download(COCO_OUTPUT_PATH)